# Optimización de modelos

## Imports

Importamos las librerías y parámetros que necesitaremos en la ejecución del Notebook.

In [1]:
import sys
sys.path.append("..")
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from datetime import datetime
from mgmtfm import optimize
import matplotlib.pyplot as plt


pd.set_option('max_rows',9999)
pd.set_option('max_columns', 9999)
pd.set_option('display.max_colwidth', 500)

OPT_BBDD="mgm_optuna"

print("Notebook ejecutado el {}.".format(datetime.now().strftime("%d-%m-%Y")))

Using TensorFlow backend.


Notebook ejecutado el 14-11-2019.


## Carga datos

Cargamos todos los estudios con la ayuda de la clase Optimize del módulo mgmtfm que ataca directamente a la BBDD.

In [2]:
opt_keras = optimize.Optimize(project_db=OPT_BBDD)
st = opt_keras.get_studies()


[I 2019-11-14 19:43:07,411] Using an existing study with name 'dummy' instead of creating a new one.


Antes del análisis vemos las horas totales que hemos usado optimizando modelos sobre nuestras Tesla V100-SXM2 32GB. 
<BR>

Hay que tener en cuenta que muchas de las pruebas no prometedoras Optuna las ha cancelado automáticamente, por lo que en condiciones normales hubieramos necesitado bastante más tiempo. También hay momentos en los que se ha paralelizado el entrenamiento entre varias GPUs.

In [3]:
str(st.sum()["total_time"])

'14 days 00:51:53.300427'

## Análisis resultados

A continuación vamos a analizar los resultados obtenidos con Optuna al entrenar los diferentes modelos. Los nombres de los modelos nos darán información sobre el tipo del modelo. Algunos datos que tenemos que tener en cuenta son:

<ul>
 <li><b>CNN/RNN/RNN2/DENSE</b>: Nos indica el tipo del modelo y estará al comienzo del nombre.</li>
 <li><b>IVR/MONI</b>: Nos indica si el modelo ha sido entrenado y validado con los tipos de IVR o de monitorizaciones. De momento solo usamos IVR</li>   
 <li><b>W2V150QC</b>: Nos indica que utiliza un Word2Vec con una dimensión de 150 y quitando las palabras comunes en los tokens.</li>
  <li><b>BIN/MULTI</b>: Nos indica que se trata de un modelo binario o multiclase. Si es binario al final del nombre nos encontramos la clase, si es multi y no clasifica todas las clases nos encontramos las iniciales de las clases.</li>
  <li><b>QX</b>: Nos indica las clases que se eliminan. Por ejemplo QR_QN elimina las clases resto y no reconocido antes de entrenar. La no reconocido la eliminamos siempre ya que puede contener varias clases.</li>
  <li><b>866</b>: Indica que en estos modelos se ha limitado las secuencias a 866 tokens, que engloban más del 99% de las llamadas.</li>
   
<ul>
    

### CNN

Aunque se han hecho pruebas con modelos multi-clase, cuando se ha empezado a utilizar Optuna unicamente se han entrenado modelos binarios que presentaban mucho mejores resultados. Los modelos se han entrenado teniendo el mismo número de muestras de la clase binaria y del resto de clases.

In [4]:

df = st[st["study"].str.contains("CNN") & st["study"].str.contains("W2V150QC")]
df.sort_values(by=["accuracy"], ascending=False)

,study,accuracy,params,ntrials,total_time,best_trial
19,CNN_IVR_BIN_W2V150QC_QR_QN_BAJA,0.862927,"{(epochs, 14.0), (kernel_size_conv_1, 2.0), (lr, 0.0466210906487132), (decay, 0.0973688009931105), (batch_size, 190.0), (num_filters, 141.0), (dropout, 0.452728938708042), (kernel_size_conv_3, 5.0), (kernel_size_conv_2, 6.0)}",28,0 days 02:00:11.240753,7163
8,CNN_866_IVR_BIN_W2V150QC_QN_BAJA,0.857002,"{(epochs, 15.0), (kernel_size_conv_3, 7.0), (lr, 0.0783251859934289), (num_filters, 116.0), (decay, 0.08655927910488), (kernel_size_conv_1, 4.0), (kernel_size_conv_2, 4.0), (batch_size, 179.0), (dropout, 0.0856431000749515)}",87,0 days 03:01:23.436193,17765
12,CNN_IVR_BIN_W2V150QC_BAJA,0.844259,"{(kernel_size_conv_1, 3.0), (kernel_size_conv_3, 7.0), (lr, 0.0706824812409031), (batch_size, 130.0), (num_filters, 138.0), (decay, 0.0817036289265317), (dropout, 0.465363078907312), (epochs, 13.0), (kernel_size_conv_2, 6.0)}",157,1 days 03:25:09.192230,428
9,CNN_866_IVR_BIN_W2V150QC_QN_FACTURA,0.837989,"{(epochs, 15.0), (kernel_size_conv_3, 7.0), (kernel_size_conv_2, 3.0), (num_filters, 180.0), (decay, 0.0677721044360595), (kernel_size_conv_1, 4.0), (batch_size, 255.0), (dropout, 0.0525838716431868), (lr, 0.0308687095584748)}",67,0 days 02:58:44.532765,17630
20,CNN_IVR_BIN_W2V150QC_QR_QN_FACTURA,0.834412,"{(kernel_size_conv_1, 3.0), (batch_size, 283.0), (kernel_size_conv_3, 7.0), (lr, 0.0689525683093965), (decay, 0.0837186209217048), (num_filters, 140.0), (kernel_size_conv_2, 5.0), (epochs, 12.0), (dropout, 0.141975383278602)}",11,0 days 02:03:49.792338,7180
7,CNN_IVR_BIN_W2V150QC_FACTURA,0.833933,"{(kernel_size_conv_3, 6.0), (decay, 0.0621436033315488), (kernel_size_conv_1, 2.0), (lr, 0.0754551969975402), (dropout, 0.0601620859900921), (batch_size, 476.0), (epochs, 5.0), (kernel_size_conv_2, 5.0), (num_filters, 99.0)}",18,0 days 02:01:35.712605,547
10,CNN_866_IVR_BIN_W2V150QC_QN_AVERÍA,0.805820,"{(decay, 0.05700552618666), (kernel_size_conv_3, 7.0), (num_filters, 125.0), (lr, 0.0663904473012762), (batch_size, 328.0), (kernel_size_conv_1, 4.0), (kernel_size_conv_2, 4.0), (epochs, 17.0), (dropout, 0.286388351609826)}",165,0 days 03:00:18.650700,17588
17,CNN_IVR_BIN_W2V150QC_QR_QN_AVERÍA,0.797523,"{(epochs, 14.0), (kernel_size_conv_1, 3.0), (kernel_size_conv_3, 6.0), (lr, 0.0548823090135392), (kernel_size_conv_2, 3.0), (num_filters, 94.0), (decay, 0.0988380792860522), (dropout, 0.329528601396929), (batch_size, 172.0)}",69,0 days 02:01:26.599629,7271
39,CNN_866_IVR_BIN_W2V150QC_QN_COMERCIAL,0.775623,"{(lr, 0.0755546150840543), (batch_size, 136.0), (kernel_size_conv_1, 5.0), (decay, 0.0868261840964118), (epochs, 9.0), (kernel_size_conv_2, 5.0), (kernel_size_conv_3, 4.0), (dropout, 0.122009867836421), (num_filters, 123.0)}",24,0 days 03:11:02.989281,19246
21,CNN_IVR_BIN_W2V150QC_QR_QN_COMERCIAL,0.775587,"{(epochs, 14.0), (kernel_size_conv_3, 7.0), (lr, 0.0651886911467839), (kernel_size_conv_2, 3.0), (num_filters, 94.0), (decay, 0.0434100642097045), (kernel_size_conv_1, 4.0), (batch_size, 486.0), (dropout, 0.378186034581814)}",4,0 days 02:16:05.047722,7200


Podemos ver resultados buenos para la mayoría de clases que van desde el 72% al 86%. Cuando eliminamos las clases resto y no reconocido vemos que los datos mejoran.

### RNN y RNN2 Binarios

En estos casos nos encontramos con una ligera mejora en algunas clases, aunque los modelos requieren bastante más tiempo de entrenamiento 

In [5]:
df = st[st["study"].str.contains("RNN") & st["study"].str.contains("BIN")]
df.sort_values(by=["accuracy"], ascending=False)

,study,accuracy,params,ntrials,total_time,best_trial
24,RNN_IVR_BIN_W2V150QC_QR_QN_BAJA,0.873912,"{(memory_units, 51.0), (batch_size, 430.0), (lr, 0.0530475544836823), (cell, 1.0), (decay, 0.0675610167878156), (epochs, 4.0), (dropout, 0.457826302587858)}",48,07:02:19.799293,10927
37,RNN2_IVR_BIN_W2V150QC_QR_QN_BAJA,0.865334,"{(filters, 59.0), (batch_size, 744.0), (memory_units, 389.0), (cell, 0.0), (decay, 0.0767493546430331), (padding, 1.0), (kernel_size, 3.0), (lr, 0.0392491376088012), (pool_size, 4.0), (dropout, 0.235471010425368), (epochs, 13.0)}",41,05:04:49.337967,11505
23,RNN_IVR_BIN_W2V150QC_QR_QN_FACTURA,0.858087,"{(decay, 0.0563279925787701), (memory_units, 274.0), (cell, 1.0), (lr, 0.0126883161110917), (batch_size, 581.0), (epochs, 12.0), (dropout, 0.43667104698883)}",5,09:13:43.006154,11063
1,RNN_866_IVR_BIN_W2V150QC_QN_FACTURA,0.854054,"{(epochs, 14.0), (lr, 0.0750196610781145), (cell, 1.0), (memory_units, 36.0), (decay, 0.0690200826771197), (dropout, 0.121860865166101), (batch_size, 185.0)}",29,07:40:09.109754,19200
35,RNN2_IVR_BIN_W2V150QC_QR_QN_FACTURA,0.848858,"{(kernel_size, 5.0), (batch_size, 469.0), (pool_size, 2.0), (memory_units, 303.0), (cell, 0.0), (decay, 0.0769827807899012), (padding, 1.0), (lr, 0.0418375676518619), (epochs, 7.0), (dropout, 0.358486430661762), (filters, 23.0)}",126,14:41:51.247073,11814
38,RNN2_IVR_BIN_W2V150QC_QR_QN_AVERÍA,0.797152,"{(kernel_size, 7.0), (lr, 0.0378537983014233), (pool_size, 3.0), (decay, 0.0270850530925684), (padding, 0.0), (batch_size, 380.0), (cell, 0.0), (memory_units, 87.0), (epochs, 11.0), (dropout, 0.179448066463683), (filters, 23.0)}",582,16:53:32.080155,11673
32,RNN2_IVR_BIN_W2V150QC_QR_QN_COMERCIAL,0.795804,"{(filters, 33.0), (kernel_size, 4.0), (cell, 0.0), (batch_size, 765.0), (epochs, 10.0), (decay, 0.0119204339719964), (memory_units, 343.0), (padding, 0.0), (pool_size, 4.0), (dropout, 0.251991845650367), (lr, 0.018451764339489)}",5,05:18:37.171854,11735
0,RNN_866_IVR_BIN_W2V150QC_QN_AVERÍA,0.789845,"{(decay, 0.0103630280252387), (cell, 1.0), (epochs, 4.0), (lr, 0.0156142046274686), (batch_size, 659.0), (dropout, 0.493818845486034), (memory_units, 170.0)}",446,06:58:10.432142,19208
4,RNN_866_IVR_BIN_W2V150QC_QN_COMERCIAL,0.771328,"{(decay, 0.0524916679520997), (epochs, 15.0), (cell, 1.0), (batch_size, 528.0), (memory_units, 66.0), (dropout, 0.102299572623736), (lr, 0.0201976677768787)}",33,20:13:44.980418,19897
25,RNN_IVR_BIN_W2V150QC_QR_QN_RECLAMACIÓN,0.761911,"{(batch_size, 661.0), (cell, 1.0), (lr, 0.0169383441457147), (decay, 0.0782390568010709), (epochs, 7.0), (dropout, 0.359685372530682), (memory_units, 127.0)}",13,04:05:32.114764,11485


Vemos por ejemplo como en el caso de factura hemos mejorado el modelo hasta casi el 86% con una red recurrente y el caso de comercial con una red recurrente con una capa convolucional hemos llegado al 79.6%.

### DENSE

Veamos el comportamiento con Doc2Vec y una capa totalmente conectada.

In [6]:
df = st[st["study"].str.contains("DENSE")]
df.sort_values(by=["accuracy"], ascending=False)

,study,accuracy,params,ntrials,total_time,best_trial
27,DENSE_IVR_BIN_D2V500w7m2_RECLAMACIÓN,0.725599,"{(decay, 0.0377856747431033), (epochs, 10.0), (dropout, 0.298847478196375), (lr, 0.0593762691588953), (batch_size, 207.0)}",762,01:53:49.283701,4799
28,DENSE_IVR_BIN_D2V500w7m2_COMERCIAL,0.692321,"{(batch_size, 469.0), (dropout, 0.0872092559873194), (lr, 0.0188969464466535), (decay, 0.00128187964520694), (epochs, 9.0)}",1517,04:47:27.896225,5075


Hemos hecho la prueba con unicamente dos clases, con un doc2vec entrenado con todo nuestro corpus y una dimensión de 500. Podemos ver que la precisión esta muy por debajo de los otros modelos.

### Multiclase

Los modelos multiclase a las que hemos aplicado el optimizador son modelos con Redes Recurrentes (RNN y RNN2); con el modelo *stacked* las pruebas que hemos realizado son menos prometedoras.

<BR>
    
Las últimas iniciales nos indican las clases que el modelo es capaz de distinguir, si acaba en *_* el modelo clasifica entre todas las clases.

In [7]:
df = st[st["study"].str.contains("MUL")]
df.sort_values(by=["accuracy"], ascending=False)

,study,accuracy,params,ntrials,total_time,best_trial
13,RNN_866_IVR_MULALL_W2V150QC_QN_BF,0.810985,"{(decay, 0.0332287311530309), (epochs, 15.0), (cell, 1.0), (memory_units, 149.0), (lr, 0.0130772443848036), (batch_size, 486.0), (dropout, 0.11319879246438)}",29,05:04:55.720171,20368
15,RNN_866_IVR_MULALL_W2V150QC_QN_BC,0.739601,"{(decay, 0.0328426707805091), (lr, 0.0414983886251781), (cell, 1.0), (batch_size, 215.0), (epochs, 11.0), (dropout, 0.191091313671187), (memory_units, 42.0)}",192,15:06:43.790940,20646
14,RNN_866_IVR_MULALL_W2V150QC_QN_CF,0.721566,"{(decay, 0.0455766288716424), (batch_size, 923.0), (lr, 0.067137891794526), (cell, 1.0), (epochs, 12.0), (dropout, 0.273525299277387), (memory_units, 43.0)}",114,23:58:15.661986,20379
5,RNN_866_IVR_MULALL_W2V150QC_QN_BCF,0.683083,"{(memory_units, 39.0), (cell, 1.0), (batch_size, 682.0), (decay, 0.0629939815112358), (epochs, 9.0), (lr, 0.0124448720550143), (dropout, 0.13267072561035)}",114,06:57:08.013540,20497
16,RNN_866_IVR_MULALL_W2V150QC_QN_CR,0.643817,"{(lr, 0.0982570484018716), (cell, 1.0), (decay, 0.0632519386277805), (batch_size, 944.0), (dropout, 0.212708265102609), (epochs, 13.0), (memory_units, 31.0)}",203,09:58:49.809159,20639
3,RNN_866_IVR_MULALL_W2V150QC_QN_,0.612574,"{(decay, 0.0564302421625528), (lr, 0.0118536116122586), (cell, 1.0), (memory_units, 100.0), (dropout, 0.146082920845253), (epochs, 13.0), (batch_size, 876.0)}",100,06:02:53.511342,20151
6,RNN2_866_IVR_MULALL_W2V150QC_QN_,0.491579,"{(lr, 0.0563633882292892), (kernel_size, 5.0), (pool_size, 3.0), (filters, 45.0), (cell, 1.0), (memory_units, 205.0), (decay, 0.0149661429365363), (padding, 1.0), (epochs, 11.0), (batch_size, 371.0), (dropout, 0.196451369147809)}",72,04:59:21.213054,20292
40,RNN_866_IVR_MULALL_W2V150QC_QN_AC,0.475396,"{(batch_size, 1024.0), (memory_units, 38.0), (epochs, 2.0), (cell, 1.0), (decay, 0.0276762850659422), (dropout, 0.173462689227962), (lr, 0.0106123552430217)}",279,07:57:48.110430,20672


Vemos que la mejor precisión en un modelo multiclase la tenemos hasta el momento en un modelo con redes recurrentes que distingue entre Bajas, Facturas y el resto de llamadas con una precisión del 81%.